<a href="https://colab.research.google.com/github/benihime91/retinanet_pet_detector/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
<h1><b> Pet Face Detector 👁 🐶 🐱 </b></h1>
<p align="center"><img src="https://github.com/benihime91/retinanet_pet_detector/blob/master/images/res_3.png?raw=true" height="300"> </p>  
<b> Using a RetinaNet to detect faces of common breeds of Pets </b>
</div>

In [ ]:
# Ensure colab doesn't disconnect
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

In [ ]:
# what GPU do we have ?
!nvidia-smi

## **Setup Google-Colab:**

In [ ]:
# install dependencies
!pip install --upgrade pytorch-lightning omegaconf --quiet
!pip install git+https://github.com/albumentations-team/albumentations --quiet

In [ ]:
# Uncommment & run this cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# clone the github repo:
!git clone --recurse-submodules https://github.com/benihime91/retinanet_pet_detector.git

**Untar the data from Google Drive and save it to VM memory :**

- The cell below unzips the data assuming the `The Oxford-IIIT Pet Dataset` is present in `/content/drive/My Drive/Data/oxford-iiit-pet.tgz.`

- If data is not downloaded download the data from [here](https://www.robots.ox.ac.uk/~vgg/data/pets) and save in it in Google Drive under `/Data/oxford-iiit-pet.tgz`.

In [ ]:
# unpacks the data
!tar xf /content/drive/My\ Drive/Data/oxford-iiit-pet.tgz -C /content/

## **Import dependencies:**

In [ ]:
import sys
import os
import warnings

os.chdir("/content/retinanet_pet_detector")
warnings.filterwarnings("ignore")
%matplotlib inline
%load_ext tensorboard

In [ ]:
# Standard Imports
import numpy as np
from omegaconf import OmegaConf, DictConfig
import pandas as pd
import argparse
from PIL import Image

from references import Visualizer
from references.utils import get_label_dict
from train import main

from google.colab import files

pd.set_option("display.max_colwidth",None)

## **Preprare Data:**

Before training we need to convert the data into a format that is compatible with the `training pipeline.` We will use `references/data_utils.py` to convert all the xml annotation files into a csv that stores all the annotations and path to the Images.

The resutant csv file will be saved as `/{ouptut_dir}/data-full.csv`.

In [ ]:
# Convert xml files to a csv file
!python prep_data.py \
    --action create \
    --img_dir "/content/oxford-iiit-pet/images" \
    --annot_dir "/content/oxford-iiit-pet/annotations/xmls" \
    --labels "/content/retinanet_pet_detector/data/labels.names" \
    --output_dir "/content/retinanet_pet_detector/data/"

This is what our data looks like:

In [ ]:
df = pd.read_csv("/content/retinanet_pet_detector/data/data-full.csv")
df.head(5)

In [ ]:
# sanity-check
Image.open(df.filename[100])

We will again the run the script used above but this time we will run this script to create `training`, `validation` & `test` sets from the full dataset.

In [ ]:
# Create train, validaiton and test splits in the data
!python prep_data.py \
    --action split \
    --csv "/content/retinanet_pet_detector/data/data-full.csv"\
    --valid_size 0.3 \
    --test_size 0.5 \
    --output_dir "/content/retinanet_pet_detector/data/" \
    --seed 123

We can see that now we have 3 extra files train.csv, valid.csv & test.csv. This files correspond to the `train`,`validation` & `test` datasets respectively.

In [ ]:
! ls "/content/retinanet_pet_detector/data"

In [ ]:
# Read in the train and test dataframes
trn_df = pd.read_csv("/content/retinanet_pet_detector/data/train.csv")
tst_df = pd.read_csv("/content/retinanet_pet_detector/data/test.csv")
val_df = pd.read_csv("/content/retinanet_pet_detector/data/valid.csv")

In [ ]:
print("Num training examples :", len(trn_df))
trn_df.head()

In [ ]:
print("Num testing examples :", len(tst_df))
tst_df.head()

In [ ]:
print("Num validation examples :", len(val_df))
val_df.head()

**View some images from the datasets:**

In [ ]:
# Read in the Labes dictionary 
# and initializer the visualizer to view images with bboxes
label_dict = get_label_dict("/content/retinanet_pet_detector/data/labels.names")
vis = Visualizer(label_dict)

In [ ]:
# These are the label
label_dict

In [ ]:
np.random.seed(123)

def display_random_image(data):
    """
    Fn to display a random using the `Visualizer`
    from the given pandas dataframe. The bounding boxes are also
    drawn over the image.

    Args:
     data (`pd.dataframe`): A `pandas dataframe` where filename corresponds to 
                            the image path and bbox co-ordinates are stored in 
                            `[xmin,xmax,ymin,ymax]` & class_names (`int`) are 
                            stored in `[labels]`.
    """
    idx = np.random.randint(0, len(df))
    image_id = df.filename[idx]
    locs = df.loc[df.filename == image_id]
    boxes = locs[['xmin','ymin','xmax','ymax']].values
    labels = locs['labels'].values
    vis.draw_bboxes(image_id, boxes, labels)

In [ ]:
# Display a random Image from the Train dataset
display_random_image(data=trn_df)

In [ ]:
# Display a random Image from the test dataset
display_random_image(data=tst_df)

In [ ]:
# Display a random Image from the validation dataset
display_random_image(data=val_df)

## **Modify Config file:**

**Setup paths:**

In [ ]:
import time

# load the config file
config = OmegaConf.load("/content/retinanet_pet_detector/config/main.yaml")

# -------------------------------------------- #
# Modify some config parameters:
# -------------------------------------------- #
fname = f"/content/drive/My Drive/{time.strftime('[%m-%d]%H-%M-%S')}"

config.model.backbone_kind = "resnet50" 
config.hparams.train_csv = "/content/retinanet_pet_detector/data/train.csv"
config.hparams.valid_csv = "/content/retinanet_pet_detector/data/valid.csv"
config.hparams.test_csv  = "/content/retinanet_pet_detector/data/test.csv"
config.trainer.model_checkpoint.params.filepath = fname
config.trainer.logger.params.save_dir = "/content/logs/"
config.trainer.early_stopping.params.patience = 20

**update config file for train:**

In [ ]:
# Save the modified config file
OmegaConf.save(config=config, f="/content/retinanet_pet_detector/config/main.yaml")

In [ ]:
# Load and view the modified config file
config = OmegaConf.load("/content/retinanet_pet_detector/config/main.yaml")
# let's take a look at out config file
print(OmegaConf.to_yaml(config))

## **Train, Validation & Test :**

In colab use the main function to train otherwise Lightning progress bar goes mad, this issue is highlighted [here]( https://github.com/PyTorchLightning/pytorch-lightning/issues/721). 

The `main` function accepts `argparse` arguments so we will first define a `Dictionary` with the args and convert it to `argparse.Namespace` instance.





**Config for train:**

In [ ]:
# Creat argument dictionary
d = {"config": "/content/retinanet_pet_detector/config/main.yaml", "verbose": 0}
# Creat argument dictionary
args = DictConfig(d)
print(args.pretty())

### **Start Loop for train, validation and test :**

In [ ]:
# run the main function
# set a seed number to ensure results are reproducible
main(args, seed=123)

In [ ]:
%tensorboard --logdir "/content/logs"

## **Inference with saved weights:**

To do inference we need to modify or create a config file for inference. The config file for inference should contain the following:
- `model_backbone` (`str`) : resnet backbone used for the retinanet model.
- `url` (`str`) : url or the path to where weights are saved.
- `num_classes` (`int`) : total number of unique classes.

We will save this `config` file at : `/content/retinanet_pet_detector/config/resnet50.yaml`

### **Instantiate config for inference:**

In [ ]:
# we used a resnet34 model so,
# we will set the backbone to be `renet34`
backbone = "resnet50"
# path to where model weights are saved
url = "/content/drive/My Drive/pets/weights.pth"
# total number of classes
num_classes = 37

d = {"model_backbone": backbone, "url": url, "num_classes": num_classes}
conf = DictConfig(d)

# Save the config File
fname = "/content/retinanet_pet_detector/config/resnet50.yaml"
OmegaConf.save(config=conf, f=fname)

# Print out the config File
print(OmegaConf.to_yaml(conf))

In [ ]:
# path to the ocnfig file
config = "/content/retinanet_pet_detector/config/resnet50.yaml"

### **Run Inference:**

In [ ]:
!python inference.py \
    --config {config} \
    --image {tst_df.filename[100]} \
    --score_thres 0.7 \
    --iou_thres 0.4 \
    --save_dir "/content/" \
    --fname "res_1.png" \

In [ ]:
Image.open("/content/res_1.png")

In [ ]:
!python inference.py \
    --config {config} \
    --image {tst_df.filename[20]} \
    --score_thres 0.7 \
    --iou_thres 0.4 \
    --save_dir "/content/" \
    --fname "res_2.png" \

In [ ]:
Image.open("/content/res_2.png")

**The following cells run inference on user uploaded image:**

In [ ]:
uploaded = files.upload()
fname = list(uploaded.keys())[0]

In [ ]:
!python inference.py \
    --config {config}\
    --image {fname} \
    --score_thres 0.7 \
    --iou_thres 0.4 \
    --save_dir "/content/" \
    --fname "res_3.png" \

In [ ]:
Image.open("/content/res_3.png")